In [5]:
from copy import deepcopy
import random
from qibo import gates
from qibo.models.circuit import Circuit
from qibo.transpiler.abstract import Router
from qibo.transpiler.router import CircuitMap, _create_dag
import networkx as nx
from typing import Optional

# Looping Behavior in the SABRE Router

### Problem
- In certain circuits, the SABRE router exhibits looping behavior.
- It repeatedly applies similar SWAP patterns without making progress.
- Qiskit also has the same issue. [Reference](https://github.com/Qiskit/qiskit/blob/e5ee1a8d97753ccca09b0c15be7b7ab0c1c84b90/crates/accelerate/src/sabre/route.rs#L559)

### Solution
- Track #SWAPs applied since the last block execution.
- If #SWAPs exceeds a defined threshold, remove the SWAPs and manually route to the nearest gate.

## Previous Sabre implementation

- Modified SABRE has been updated in Qibo.

In [8]:
class Sabre_previous(Router):
    """Routing algorithm proposed in Ref [1].

    Args:
        connectivity (:class:`networkx.Graph`): hardware chip connectivity.
        lookahead (int, optional): lookahead factor, how many dag layers will be considered
            in computing the cost. Defaults to :math:`2`.
        decay_lookahead (float, optional): value in interval :math:`[0, 1]`.
            How the weight of the distance in the dag layers decays in computing the cost.
            Defaults to :math:`0.6`.
        delta (float, optional): defines the number of SWAPs vs depth trade-off by deciding
            how the algorithm tends to select non-overlapping SWAPs.
            Defaults to math:`10^{-3}`.
        seed (int, optional): seed for the candidate random choice as tiebraker.
            Defaults to ``None``.

    References:
        1. G. Li, Y. Ding, and Y. Xie,
        *Tackling the Qubit Mapping Problem for NISQ-Era Quantum Devices*.
        `arXiv:1809.02573 [cs.ET] <https://arxiv.org/abs/1809.02573>`_.
    """

    def __init__(
        self,
        connectivity: nx.Graph,
        lookahead: int = 2,
        decay_lookahead: float = 0.6,
        delta: float = 0.001,
        seed: Optional[int] = None,
    ):
        self.connectivity = connectivity
        self.lookahead = lookahead
        self.decay = decay_lookahead
        self.delta = delta
        self._delta_register = None
        self._dist_matrix = None
        self._dag = None
        self._front_layer = None
        self.circuit = None
        self._memory_map = None
        self._final_measurements = None
        random.seed(seed)

    def __call__(self, circuit: Circuit, initial_layout: dict):
        """Route the circuit.

        Args:
            circuit (:class:`qibo.models.circuit.Circuit`): circuit to be routed.
            initial_layout (dict): initial physical to logical qubit mapping.

        Returns:
            (:class:`qibo.models.circuit.Circuit`, dict): routed circuit and final layout.
        """
        self._preprocessing(circuit=circuit, initial_layout=initial_layout)
        while self._dag.number_of_nodes() != 0:
            execute_block_list = self._check_execution()
            if execute_block_list is not None:
                self._execute_blocks(execute_block_list)
            else:
                self._find_new_mapping()

        circuit_kwargs = circuit.init_kwargs
        circuit_kwargs["wire_names"] = list(initial_layout.keys())
        routed_circuit = self.circuit.routed_circuit(circuit_kwargs=circuit_kwargs)
        if self._final_measurements is not None:
            routed_circuit = self._append_final_measurements(
                routed_circuit=routed_circuit
            )

        return routed_circuit, self.circuit.final_layout()

    @property
    def added_swaps(self):
        """Returns the number of SWAP gates added to the circuit during routing."""
        return self.circuit._swaps

    def _preprocessing(self, circuit: Circuit, initial_layout: dict):
        """The following objects will be initialised:
            - circuit: class to represent circuit and to perform logical-physical qubit mapping.
            - _final_measurements: measurement gates at the end of the circuit.
            - _dist_matrix: matrix reporting the shortest path lengh between all node pairs.
            - _dag: direct acyclic graph of the circuit based on commutativity.
            - _memory_map: list to remember previous SWAP moves.
            - _front_layer: list containing the blocks to be executed.
            - _delta_register: list containing the special weigh added to qubits
                to prevent overlapping swaps.

        Args:
            circuit (:class:`qibo.models.circuit.Circuit`): circuit to be preprocessed.
            initial_layout (dict): initial physical-to-logical qubit mapping.
        """
        copied_circuit = circuit.copy(deep=True)
        self._final_measurements = self._detach_final_measurements(copied_circuit)
        self.circuit = CircuitMap(initial_layout, copied_circuit)
        self._dist_matrix = nx.floyd_warshall_numpy(self.connectivity)
        self._dag = _create_dag(self.circuit.blocks_qubits_pairs())
        self._memory_map = []
        self._update_dag_layers()
        self._update_front_layer()
        self._delta_register = [1.0 for _ in range(circuit.nqubits)]

    def _detach_final_measurements(self, circuit: Circuit):
        """Detach measurement gates at the end of the circuit for separate handling."""
        final_measurements = []
        for gate in circuit.queue[::-1]:
            if isinstance(gate, gates.M):
                final_measurements.append(gate)
                circuit.queue.remove(gate)
            else:
                break
        if not final_measurements:
            return None
        return final_measurements[::-1]

    def _append_final_measurements(self, routed_circuit: Circuit):
        """Appends final measurment gates on the correct qubits conserving the measurement register.

        Args:
            routed_circuit (:class:`qibo.models.circuit.Circuit`): original circuit.

        Returns:
            (:class:`qibo.models.circuit.Circuit`) routed circuit.
        """
        for measurement in self._final_measurements:
            original_qubits = measurement.qubits
            routed_qubits = list(
                self.circuit.circuit_to_physical(qubit) for qubit in original_qubits
            )
            routed_circuit.add(
                measurement.on_qubits(dict(zip(original_qubits, routed_qubits)))
            )
        return routed_circuit

    def _update_dag_layers(self):
        """Update dag layers and put them in topological order.

        Method works in-place.
        """
        for layer, nodes in enumerate(nx.topological_generations(self._dag)):
            for node in nodes:
                self._dag.nodes[node]["layer"] = layer

    def _update_front_layer(self):
        """Update the front layer of the dag.

        Method works in-place.
        """
        self._front_layer = self._get_dag_layer(0)

    def _get_dag_layer(self, n_layer):
        """Return the :math:`n`-topological layer of the dag."""
        return [node[0] for node in self._dag.nodes(data="layer") if node[1] == n_layer]

    def _find_new_mapping(self):
        """Find the new best mapping by adding one swap."""
        candidates_evaluation = {}
        self._memory_map.append(deepcopy(self.circuit._circuit_logical))
        for candidate in self._swap_candidates():
            candidates_evaluation[candidate] = self._compute_cost(candidate)

        best_cost = min(candidates_evaluation.values())
        best_candidates = [
            key for key, value in candidates_evaluation.items() if value == best_cost
        ]
        best_candidate = random.choice(best_candidates)

        for qubit in self.circuit.logical_to_physical(best_candidate, index=True):
            self._delta_register[qubit] += self.delta
        self.circuit.update(best_candidate)

    def _compute_cost(self, candidate: int):
        """Compute the cost associated to a possible SWAP candidate."""
        temporary_circuit = CircuitMap(
            initial_layout=self.circuit.initial_layout,
            circuit=Circuit(len(self.circuit.initial_layout)),
            blocks=self.circuit.circuit_blocks,
        )
        temporary_circuit.set_circuit_logical(deepcopy(self.circuit._circuit_logical))
        temporary_circuit.update(candidate)

        if temporary_circuit._circuit_logical in self._memory_map:
            return float("inf")

        tot_distance = 0.0
        weight = 1.0
        for layer in range(self.lookahead + 1):
            layer_gates = self._get_dag_layer(layer)
            avg_layer_distance = 0.0
            for gate in layer_gates:
                qubits = temporary_circuit.get_physical_qubits(gate, index=True)
                avg_layer_distance += (
                    max(self._delta_register[i] for i in qubits)
                    * (self._dist_matrix[qubits[0], qubits[1]] - 1.0)
                    / len(layer_gates)
                )
            tot_distance += weight * avg_layer_distance
            weight *= self.decay

        return tot_distance

    def _swap_candidates(self):
        """Returns a list of possible candidate SWAPs to be applied on logical qubits directly.

        The possible candidates are the ones sharing at least one qubit
        with a block in the front layer.

        Returns:
            (list): list of candidates.
        """
        candidates = []
        for block in self._front_layer:
            for qubit in self.circuit.get_physical_qubits(block):
                for connected in self.connectivity.neighbors(qubit):
                    candidate = tuple(
                        sorted(
                            (
                                self.circuit.physical_to_logical(qubit),
                                self.circuit.physical_to_logical(connected),
                            )
                        )
                    )
                    if candidate not in candidates:
                        candidates.append(candidate)

        return candidates

    def _check_execution(self):
        """Check if some blocks in the front layer can be executed in the current configuration.

        Returns:
            (list): executable blocks if there are, ``None`` otherwise.
        """
        executable_blocks = []
        for block in self._front_layer:
            if (
                self.circuit.get_physical_qubits(block) in self.connectivity.edges
                or not self.circuit.circuit_blocks.search_by_index(block).entangled
            ):
                executable_blocks.append(block)

        if len(executable_blocks) == 0:
            return None

        return executable_blocks

    def _execute_blocks(self, blocklist: list):
        """Executes a list of blocks:
        -Remove the correspondent nodes from the dag and circuit representation.
        -Add the executed blocks to the routed circuit.
        -Update the dag layers and front layer.
        -Reset the mapping memory.

        Method works in-place.

        Args:
            blocklist (list): list of blocks.
        """
        for block_id in blocklist:
            block = self.circuit.circuit_blocks.search_by_index(block_id)
            self.circuit.execute_block(block)
            self._dag.remove_node(block_id)
        self._update_dag_layers()
        self._update_front_layer()
        self._memory_map = []
        self._delta_register = [1.0 for _ in self._delta_register]

### Utils

In [9]:
def count_swaps_qibo(circuit):
    count = 0
    for gate in circuit.queue:
        if isinstance(gate, gates.SWAP):
            count += 1
    return count

## Problem

- Circuits with 6 CZs on 10 qubits
- **\>100 SWAPS** are added

In [33]:
from qibo.cskim_utils.connectivity import line_connectivity_nx
from qibo.transpiler.placer import Random, Trivial
from qibo.transpiler.router import Sabre

nqubits = 10
qubit_array = [(7, 2), (6, 0), (5, 6), (4, 8), (3, 5), (9, 1)]

abnormal_circ = Circuit(nqubits)
for qubit in qubit_array:
    abnormal_circ.add(gates.CZ(*qubit))

conn = line_connectivity_nx(nqubits)
placer = Trivial(connectivity=conn)
initial_layout = placer(abnormal_circ)
router = Sabre_previous(connectivity=conn)      # The original Sabre algorithm
routed_circuit, final_layout = router(abnormal_circ, initial_layout=initial_layout)
print(routed_circuit.draw(line_wrap=1000))

print()
print("#SWAPS added by the original Sabre algorithm", count_swaps_qibo(routed_circuit))

q0: ─────x─────x───────────────x───────────────────────x───────x─────────────x─────────────x───────x─────────────x─────────x───────x───────────x───x───────────x───────────x───────────────x─────x───x───x─────x─────────────x─────────────────x───x─────────────────────x───────────────────────────────────────────────────────────x───
q1: ─────x───x─x─x───────x───x─x───x─────x───────x───x─x─────x─x─x───x───x───x───x─────x───x─────x─x─x───x───x───x───x─────x─x─────x───x─────x─x─x─x───────x───x───x───────x─x───x─────x───x─x───x─x─x───x───x─x─────x───────x─────x───────x───x─x─x─x───────x─────x─────x─x───────x─────────────────────────x───────────────────────x─Z─
q2: ───x─────x───x───────x─x─x───x─x─x───x───x───x─x─x───x───x───x───x─x─x─────x─x─x───x─────────x───x───x─x─x───────x───────x─x─────x─x─x───x───x─────x───x───────x─────x───x───x─x───x─x───x─────x───────x─x───x───x───x─────────x───────x─────x───x───────x─x───x───────x───────x───x─────────x───────────x─x───────────────────x───o─
q3: ─x─x──

## Fixed

- Track #SWAPs applied since the last block execution.
- If #SWAPs exceeds a defined threshold, remove the SWAPs and manually route to the nearest gate.

```py

# router.py
...
while self._dag.number_of_nodes() != 0:
    execute_block_list = self._check_execution()
    if execute_block_list is not None:
        self._execute_blocks(execute_block_list)
    else:
        self._find_new_mapping()


    # If the number of added swaps is too high, the algorithm is stuck.
    # Reset the circuit to the last saved state and make the nearest gate executable by manually adding SWAPs.
    if self._added_swaps > 1.5 * longest_path:      # threshold is arbitrary
        self.circuit = deepcopy(self._saved_circuit)
        self._route_to_nearest_gate()
...
```

In [35]:
nqubits = 10
qubit_array = [(7, 2), (6, 0), (5, 6), (4, 8), (3, 5), (9, 1)]

abnormal_circ = Circuit(nqubits)
for qubit in qubit_array:
    abnormal_circ.add(gates.CZ(*qubit))

conn = line_connectivity_nx(nqubits)
placer = Trivial(connectivity=conn)
initial_layout = placer(abnormal_circ)
router = Sabre(connectivity=conn)      # The modified Sabre algorithm
routed_circuit, final_layout = router(abnormal_circ, initial_layout=initial_layout)
print(routed_circuit.draw(line_wrap=1000))

print()
print("#SWAPS added by the modified Sabre algorithm", count_swaps_qibo(routed_circuit))

q0: ─────────────────────────────────x─────────o─
q1: ─────────────────────────x───────x─x─────o─Z─
q2: ─────────x───────────────x─x───────x─Z─x─Z───
q3: ─────────x─x───────────────x─x───────o─x─────
q4: ─x─────────x─x───────────────x─Z─────────────
q5: ─x─x─────────x─Z───────x───────o─────────────
q6: ───x─x─────────o─────x─x─────────────────────
q7: ─────x─o───────────x─x───────────────────────
q8: ───────Z─────────x─x─────────────────────────
q9: ─────────────────x───────────────────────────

#SWAPS added by the modified Sabre algorithm 16


## Additional Tests

- QFT circuits were tested.
- random CZ circuits were tested over 10,000 times using the Random placer.

![](https://raw.githubusercontent.com/csookim/qibo_transpiler_cskim/7681ea78b5ae23d6e6925bd8d1f3740179aa167f/cskim/docs/imgs/mod_sabre_30cz.png)

![](https://raw.githubusercontent.com/csookim/qibo_transpiler_cskim/7681ea78b5ae23d6e6925bd8d1f3740179aa167f/cskim/docs/imgs/org_sabre_30cz.png)